### Запуск QtLab

In [1]:
%run D:\qtlab_replacement\init.py
%matplotlib qt4
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *

In [14]:
from resonator_tools import circuit

# Спектроскопия

In [3]:
current_src = Yokogawa_GS210(address='gs210')
pna = Agilent_N5242A('pna', address = 'PNA')
lo1 = Agilent_E8257D('lo1', address = 'PSG1')

In [4]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")

True

## Резонаторная спектроскопия

In [11]:
currents = np.linspace(-3e-4, 5e-4, 801)
pna.set_centerfreq(4.75e9)
pna.set_span(1.5e9)
pna.set_bandwidth(100)
pna.set_nop(751)
pna.set_power(-50)
sweep.sweep(pna, (currents, current_src.set_current, 'Coil current'), filename='WTJPA Ic6.9uAC3.6pF spectrum')

Started at:  Apr 28 2017 19:17:33


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Coil current: 0.0005,	average cycle time:  7.5s	5s				
Elapsed time: 1 h 40 m 45.76 s


{'S-parameter': (('Coil current', 'Frequency'),
  (array([ -3.00000000e-04,  -2.99000000e-04,  -2.98000000e-04,
           -2.97000000e-04,  -2.96000000e-04,  -2.95000000e-04,
           -2.94000000e-04,  -2.93000000e-04,  -2.92000000e-04,
           -2.91000000e-04,  -2.90000000e-04,  -2.89000000e-04,
           -2.88000000e-04,  -2.87000000e-04,  -2.86000000e-04,
           -2.85000000e-04,  -2.84000000e-04,  -2.83000000e-04,
           -2.82000000e-04,  -2.81000000e-04,  -2.80000000e-04,
           -2.79000000e-04,  -2.78000000e-04,  -2.77000000e-04,
           -2.76000000e-04,  -2.75000000e-04,  -2.74000000e-04,
           -2.73000000e-04,  -2.72000000e-04,  -2.71000000e-04,
           -2.70000000e-04,  -2.69000000e-04,  -2.68000000e-04,
           -2.67000000e-04,  -2.66000000e-04,  -2.65000000e-04,
           -2.64000000e-04,  -2.63000000e-04,  -2.62000000e-04,
           -2.61000000e-04,  -2.60000000e-04,  -2.59000000e-04,
           -2.58000000e-04,  -2.57000000e-04,  -2.560000

In [13]:
powers = np.arange(-45,-15,1.)
sweep.sweep(pna, (powers, pna.set_power, 'PNA power'), filename='WTJPA Ic6.9uAC3.6pF power_nonlin')

Started at:  Apr 28 2017 21:42:36


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: PNA power:    -16,	average cycle time:  1.3s			
Elapsed time: 0 h 0 m 38.45 s


{'S-parameter': (('PNA power', 'Frequency'),
  (array([-45., -44., -43., -42., -41., -40., -39., -38., -37., -36., -35.,
          -34., -33., -32., -31., -30., -29., -28., -27., -26., -25., -24.,
          -23., -22., -21., -20., -19., -18., -17., -16.]),
   array([  4.84500000e+09,   4.84552378e+09,   4.84604756e+09,
            4.84657134e+09,   4.84709512e+09,   4.84761890e+09,
            4.84814268e+09,   4.84866646e+09,   4.84919024e+09,
            4.84971402e+09,   4.85023780e+09,   4.85076158e+09,
            4.85128536e+09,   4.85180914e+09,   4.85233292e+09,
            4.85285670e+09,   4.85338048e+09,   4.85390426e+09,
            4.85442804e+09,   4.85495181e+09,   4.85547559e+09,
            4.85599937e+09,   4.85652315e+09,   4.85704693e+09,
            4.85757071e+09,   4.85809449e+09,   4.85861827e+09,
            4.85914205e+09,   4.85966583e+09,   4.86018961e+09,
            4.86071339e+09,   4.86123717e+09,   4.86176095e+09,
            4.86228473e+09,   4.8628085

### Загрузка данных

In [8]:
filename = 'S-parameter WTJPA Ic6.9uAC3.6pF spectrum'
location = 'D:\\qtlab\\ReiData\\data\\2017-04-28\\19-17-33'
data = load_pkl(filename, location=location)[1]

D:\qtlab\ReiData\data\2017-04-28\19-17-33/S-parameter WTJPA Ic6.9uAC3.6pF spectrum.pkl


In [21]:
S21 = data['S-parameter'][2]
currents = data['S-parameter'][1][0]
frequencies = data['S-parameter'][1][1]
fitresults = []

In [22]:
for cur_id, cur in enumerate(currents):
    fitter = circuit.reflection_port(f_data=frequencies, z_data_raw=S21[cur_id,:])
    fitresults.append(fitter.autofit())

c:\python36\lib\site-packages\scipy\optimize\minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
c:\python36\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
c:\python36\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 600.
  warnings.warn(errors[info][0], RuntimeWarning)
c:\python36\lib\site-packages\resonator_tools-0.1-py3.6.egg\resonator_tools\circuit.py:132: RuntimeWarning: invalid value encountered in sqrt
  errors = np.sqrt(np.diagonal(cov))
c:\python36\lib\site-packages\resonator_tools-0.1-py3.6.egg\resonator_tools\circuit.py:137: RuntimeWarning: invalid value encountered in sqrt
  Qi_err = np.sqrt((dQl**2*cov[2][2]) + (dQc**2*cov[1][1])+(2*dQl*dQc*cov[2][1]))  #with correlations
c:\python36\lib\si

In [28]:
plt.plot(np.real(S21[0,:]))